In [1]:
#from model import *
#from data import *
# from squeezeu import *
# from squeezeunet1 import *
from squ import *

Using TensorFlow backend.


### Train your Unet with membrane data
bluepad data is in folder bluepad/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 3)

### Train with data generator In Color / Out Gray

In [2]:
img_rows = 224
img_cols = 224
channels = 3
epochs = 10
batch_size = 1
nb_train_samples = 5000
nb_validation_samples = 2000
save_to_dir = None
inputs = Input((img_rows, img_cols, channels))

# Supporting function for dataset generator functions

In [3]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

def adjustData(img,mask,flag_multi_class,num_class):
    # used  -> trainGenerator, geneTrainNpy
    # multilass 
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    
    # one class
    elif(np.max(img) > 1): 
        img = img / 255.0
        mask = mask /255.0
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    
    return (img,mask)



def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 1,save_to_dir = None,target_size = (224,224),seed = 1):
    # -> train.py
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [4]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip = True,
                    vertical_flip  =  True,
                    
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/bluepad/train','image','label',data_gen_args,save_to_dir = None)

In [5]:
model = SqueezeUNet(inputs, num_classes=1, activation='softmax')
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=1e-04), metrics=["accuracy"])


model_checkpoint = ModelCheckpoint('sqeezeU.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [6]:
%%time
model.fit_generator(myGene, steps_per_epoch=5000, epochs=10, callbacks= [model_checkpoint])

Epoch 1/10
Found 401 images belonging to 1 classes.
Found 401 images belonging to 1 classes.
5000/5000 [==============================] - 149s 30ms/step - loss: 12.5076 - acc: 0.2154

Epoch 00001: loss improved from inf to 12.50742, saving model to sqeezeU.hdf5
Epoch 2/10
5000/5000 [==============================] - 145s 29ms/step - loss: 12.5052 - acc: 0.2156

Epoch 00002: loss improved from 12.50742 to 12.50510, saving model to sqeezeU.hdf5
Epoch 3/10
5000/5000 [==============================] - 145s 29ms/step - loss: 12.5033 - acc: 0.2157

Epoch 00003: loss improved from 12.50510 to 12.50323, saving model to sqeezeU.hdf5
Epoch 4/10
5000/5000 [==============================] - 145s 29ms/step - loss: 12.5032 - acc: 0.2157

Epoch 00004: loss improved from 12.50323 to 12.50316, saving model to sqeezeU.hdf5
Epoch 5/10
5000/5000 [==============================] - 145s 29ms/step - loss: 12.5052 - acc: 0.2156

Epoch 00005: loss did not improve from 12.50316
Epoch 6/10
5000/5000 [===========

### Train with npy file

In [7]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test model and save predicted results

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
def testGenerator(test_path, num_image = 30, target_size = (244,244), flag_multi_class = False,as_gray = False):
    #  -> train.py
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d.jpg"%i), as_gray = as_gray)
        img = img / 255
        
        img = trans.resize(img,target_size)
        
        img = np.reshape(img, img.shape) if (not flag_multi_class) else img
        img = np.reshape(img, (1,) + img.shape)
        yield img

In [9]:
testGene = testGenerator("data/bluepad/test")
model = SqueezeUNet(inputs, num_classes=1, activation='softmax') #unet()
model.load_weights("sqeezeU.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test", results)

/home/wojciech/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/wojciech/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (244, 244, 3)